In [1]:
import os
import pandas
import pydub

In [2]:
path_to_dataset = "D:\COUGHVID_public_dataset\public_dataset"
files = os.listdir(path_to_dataset)
print(f"There are {len(files)} files")
extensions = set([os.path.splitext(f)[1] for f in files])
print(f"Files have extensions {extensions}")

There are 55101 files
Files have extensions {'.json', '.ogg', '.csv', '.webm'}


In [3]:
files_by_type = {}
for ext in extensions:
    files_by_type[ext] = [f for f in files if f.endswith(ext)]
    print(f"There are {len(files_by_type[ext])} files with extension {ext}")

There are 27550 files with extension .json
There are 1565 files with extension .ogg
There are 1 files with extension .csv
There are 25985 files with extension .webm


In [4]:
import librosa
import IPython

In [5]:
import numpy as np

In [6]:
def segment_cough(x,fs, cough_padding=0.2,min_cough_len=0.2, th_l_multiplier = 0.1, th_h_multiplier = 2):
    """Preprocess the data by segmenting each file into individual coughs using a hysteresis comparator on the signal power
    
    Inputs:
    *x (np.array): cough signal
    *fs (float): sampling frequency in Hz
    *cough_padding (float): number of seconds added to the beginning and end of each detected cough to make sure coughs are not cut short
    *min_cough_length (float): length of the minimum possible segment that can be considered a cough
    *th_l_multiplier (float): multiplier of the RMS energy used as a lower threshold of the hysteresis comparator
    *th_h_multiplier (float): multiplier of the RMS energy used as a high threshold of the hysteresis comparator
    
    Outputs:
    *coughSegments (np.array of np.arrays): a list of cough signal arrays corresponding to each cough
    cough_mask (np.array): an array of booleans that are True at the indices where a cough is in progress"""
                
    cough_mask = np.array([False]*len(x))
    

    #Define hysteresis thresholds
    rms = np.sqrt(np.mean(np.square(x)))
    seg_th_l = th_l_multiplier * rms
    seg_th_h =  th_h_multiplier*rms

    #Segment coughs
    coughSegments = []
    padding = round(fs*cough_padding)
    min_cough_samples = round(fs*min_cough_len)
    cough_start = 0
    cough_end = 0
    cough_in_progress = False
    tolerance = round(0.01*fs)
    below_th_counter = 0
    
    for i, sample in enumerate(x**2):
        if cough_in_progress:
            if sample<seg_th_l:
                below_th_counter += 1
                if below_th_counter > tolerance:
                    cough_end = i+padding if (i+padding < len(x)) else len(x)-1
                    cough_in_progress = False
                    if (cough_end+1-cough_start-2*padding>min_cough_samples):
                        coughSegments.append(x[cough_start:cough_end+1])
                        cough_mask[cough_start:cough_end+1] = True
            elif i == (len(x)-1):
                cough_end=i
                cough_in_progress = False
                if (cough_end+1-cough_start-2*padding>min_cough_samples):
                    coughSegments.append(x[cough_start:cough_end+1])
            else:
                below_th_counter = 0
        else:
            if sample>seg_th_h:
                cough_start = i-padding if (i-padding >=0) else 0
                cough_in_progress = True
    
    return coughSegments, cough_mask

In [7]:
import time

In [8]:
def test_segmentation_time():
    start = time.time()

    csv_file='metadata_compiled.csv'
    assert os.path.isdir(path_to_dataset), f'Data directory {data_dir} does not exist.'
    data_dir = path_to_dataset
    csv_path = os.path.join(data_dir, csv_file)
    assert os.path.isfile(csv_path), f'CSV file {csv_path} does not exist.'
    with open(csv_path, 'r') as f:
        dataframe = pandas.read_csv(f)
    audio_extensions = ['.webm', '.ogg']
    
    
    status_map = {'healthy': 0, 'symptomatic': 1, "COVID-19": 2, 'NaN': -1}
    for key, value in status_map.items():
            dataframe.loc[dataframe['status'] == key, 'status'] = value
    status = np.isin(dataframe['status'],[0,1,2])#['healthy','symptomatic','COVID-19'])
    cough_detected = dataframe['cough_detected'] > 0.8 # recommended threshold from https://www.nature.com/articles/s41597-021-00937-4

    dataframe = dataframe[ np.logical_and(status,cough_detected) ]
    

    print(f'{len(dataframe)} records ready to load.')
    
    
    
    
    
    count = 1
    total_mask = []
    
    for i in range(len(dataframe)):
        entry = dataframe.iloc[i]
        uuid = entry['uuid']
        audio = None
        for ext in audio_extensions:
            filename = os.path.join(data_dir, f'{uuid}{ext}')
            #logger.debug(filename)
            #print(filename)
            if os.path.isfile(filename):
                audio = pydub.AudioSegment.from_file(filename)
                break
        assert audio is not None, f"No audio found with uuid {uuid}"
        audio = np.array(audio.get_array_of_samples(), dtype='int64')
        cough_segment, cough_mask = segment_cough(audio, 48000)
        total_mask.append(cough_mask)
       
    np.save('segmentation.npy', total_mask, allow_pickle = True)
    end = time.time()
    total = end - start
    return total

In [9]:
test_segmentation_time()

10817 records ready to load.


D:\Anaconda\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


14756.128456354141

In [13]:
start = time.time()
data = np.load('segmentation.npy', allow_pickle = True)
end = time.time()
print(end - start)

13.243956089019775
